# 모델의 불공정성 검색 및 완화

기계 학습 모델에는 처음 의도와는 달리 편향된 정보가 포함될 수 있으며, 그러면 모델의 *공정성* 관련 문제가 발생할 수 있습니다. 가령 당뇨병 발병 가능성을 예측하는 모델이 특정 연령 그룹에서 정확하지 않은 방식으로 작동하면 일부 환자가 불필요한 검사를 받아야 하거나 당뇨병 진단을 확인할 수 있는 검사를 받지 못할 수도 있습니다.

이 Notebook에서는 **Fairlearn** 패키지를 사용해 모델을 분석하여 연령을 기준으로 구분된 여러 환자 하위 집합을 대상으로 예측 성능의 차이를 살펴봅니다.

> **참고**: 현재 Fairlearn 패키지와 Azure Machine Learning을 통합하는 기능은 미리 보기 상태입니다. 따라서 예기치 않은 오류가 발생할 수 있습니다.

## 중요 - 공정성 관련 고려 사항

> 이 Notebook에서는 Fairlearn 패키지, 그리고 이 패키지와 Azure Machine Learning을 통합하는 방법을 파악하기 위한 실제 연습을 진행할 수 있습니다. 하지만 조직 또는 데이터 과학자 팀은 이 도구를 사용하기 전에 공정성과 관련한 여러 가지 고려 사항을 논의해야 합니다. 공정성은 도구를 실행하여 모델을 분석하는 것만으로는 충족할 수 없는 복잡한 *사회 기술적* 요소이기 때문입니다.
>
> Microsoft Research에서 공동 개발한 [공정성 체크리스트](https://www.microsoft.com/en-us/research/publication/co-designing-checklists-to-understand-organizational-challenges-and-opportunities-around-fairness-in-ai/)를 참조하면 실제 코드를 작성하기 전에 논의해야 하는 중요한 사항을 손쉽게 파악할 수 있습니다.

## 필수 SDK 설치

Azure Machine Learning에서 Fairlearn 패키지를 사용하려면 Azure Machine Learning 및 Fairlearn Python 패키지가 필요합니다. 먼저 다음 셀의 명령을 실행하여 **azureml-contrib-fairness** 패키지가 설치되어 있는지 확인합니다. 

In [ ]:
!pip show azureml-contrib-fairness

**fairlearn** 패키지 자체도 필요합니다. 다음 셀의 명령을 실행하여 패키지를 설치합니다.

In [ ]:
!pip install --upgrade fairlearn==0.5.0

## 모델 학습

먼저 당뇨병 발병 가능성을 예측하도록 분류 모델을 학습시켜야 합니다. 그러려면 데이터를 기능과 레이블이 포함된 학습 세트와 테스트 세트로 분할해야 합니다. 또한 공정성 비교 대상인 데이터의 부분 모집단을 정의하는 데 사용되는 *중요* 기능도 추출해야 합니다. 여기서는 **Age** 열을 사용하여 환자 범주 2개를 정의합니다. 즉, 전체 환자를 50세 초과 환자와 50세 이하 환자로 구분합니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# 당뇨병 데이터 세트 로드
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# 기능 및 레이블 분리
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
X, y = data[features].values, data['Diabetic'].values

# 중요한 특징 가져오기
S = data[['Age']].astype(int)
# 연령 그룹을 나타내도록 값 변경
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.20, random_state=0, stratify=y)

# 분류 모델 학습
print("Training model...")
diabetes_model = DecisionTreeClassifier().fit(X_train, y_train)

print("Model trained.")

모델을 학습시킨 후에는 Fairlearn 패키지를 사용해 각 중요 기능 값에 대한 모델의 동작을 비교할 수 있습니다. 여기서는 다음과 같은 과정을 진행합니다.

- fairlearn **selection_rate** 함수를 사용하여 전체 모집단의 선택률(당뇨병 발생 가능성을 정확하게 예측한 백분율)을 반환합니다.
- **scikit-learn** 메트릭 함수를 사용하여 전반적인 정확도, 재현율 및 정밀도 메트릭을 계산합니다.
- **MetricFrame**을 사용하여 **Age** 중요 기능 내 각 연령 그룹의 선택률, 정확도, 재현율 및 정밀도를 계산합니다. 즉, **fairlearn** 및 **scikit-learn** 메트릭 함수를 함께 사용하여 모델의 성능 값을 계산합니다.

In [ ]:
from fairlearn.metrics import selection_rate, MetricFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score

# 보류된 테스트 데이터에 대한 예측 가져오기
y_hat = diabetes_model.predict(X_test)

# 전체 메트릭 가져오기
print("Overall Metrics:")
# Fairlearn에서 선택률 가져오기
overall_selection_rate = selection_rate(y_test, y_hat) # Get selection rate from fairlearn
print("\tSelection Rate:", overall_selection_rate)
# scikit-learn에서 표준 메트릭 가져오기
overall_accuracy = accuracy_score(y_test, y_hat)
print("\tAccuracy:", overall_accuracy)
overall_recall = recall_score(y_test, y_hat)
print("\tRecall:", overall_recall)
overall_precision = precision_score(y_test, y_hat)
print("\tPrecision:", overall_precision)

# FairLearn에서 중요한 그룹별로 메트릭 가져오기
print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score}

group_metrics = MetricFrame(metrics,
                             y_test, y_hat,
                             sensitive_features=S_test['Age'])

print(group_metrics.by_group)

이러한 메트릭을 확인하면 모델이 고령층 환자의 대다수를 당뇨병 발생 대상으로 예측한다는 점을 알 수 있습니다. 두 그룹의 예측 *정확도*는 어느 정도 비슷합니다. 하지만 *정밀도*와 *재현율*을 자세히 살펴보면 모델의 각 연령 그룹 예측 성능에 다소 차이가 있음을 파악할 수 있습니다.

여기서는 *재현율*을 살펴보겠습니다. 이 메트릭은 모델이 당뇨병 발생 가능성을 정확하게 파악한 사례의 비율을 나타냅니다. 즉, 실제 당뇨병 환자 중 모델이 찾아낸 환자의 비율이 재현율인 것입니다. 환자의 연령층이 높을수록 모델의 재현율도 높습니다.

메트릭은 시각적으로 비교하면 더 쉽게 파악할 수 있는 경우가 많습니다. Fairlearn 대시보드를 사용하면 메트릭을 시각적으로 비교할 수 있습니다.

1. 아래 셀의 명령을 실행합니다. *이후 변경 사항에 대한 경고가 표시될 수도 있는데 일단은 무시하면 됩니다.*
2. 위젯이 표시되면 **시작** 링크를 사용하여 시각화 구성을 시작합니다.
3. 비교하려는 중요 기능을 선택합니다. 여기서는 중요 기능이 **Age** 하나뿐입니다.
4. 비교하려는 모델 성능 메트릭을 선택합니다. 이 모델은 이진 분류 모델이므로 *정확도*, *균형 정확도*, *정밀도*, *재현율* 중에서 선택할 수 있습니다. 먼저 **재현율**부터 선택합니다.
5. 대시보드 시각화에는 다음 정보가 표시됩니다.
    - **성능 차이** - 선택한 성능 메트릭의 부분 모집단 간 차이입니다. *과소 예측*(거짓 부정) 및 *과대 예측*(가양성)이 포함됩니다.
    - **예측률 차이** - 부분 모집단별 긍정 사례 수의 차이입니다.
6. 구성을 편집하여 다른 성능 메트릭을 기준으로 예측 결과를 비교합니다.

In [ ]:
from fairlearn.widget import FairlearnDashboard

# 이 모델을 Fairlearn의 공정성 대시보드에서 확인하고 나타내는 차이를 확인합니다.
FairlearnDashboard(sensitive_features=S_test, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test,
                   y_pred={"diabetes_model": diabetes_model.predict(X_test)})

50세가 넘는 환자 집단에서 모델의 선택률이 훨씬 높아집니다. 그러나 연령은 실제로 당뇨병 발병을 좌우한다고 할 수도 있는 요인이므로 고령층 환자에서 긍정 사례 수가 더 많아야 합니다.

모델이 정확하게 예측하는 백분율을 나타내는 *정확도*를 기준으로 모델 성능을 비교해 보면 두 부분 모집단에 대한 예측 성능은 어느 정도 비슷하다고 볼 수 있습니다. 반면 *정밀도* 및 *재현율* 메트릭을 기준으로 하면 50세가 넘는 환자의 경우에 모델의 성능이 더 우수한 편입니다.

그러면 모델을 학습시킬 때 **Age** 기능을 제외하는 경우 어떤 결과가 발생하는지 살펴보겠습니다.

In [ ]:
# 기능 및 레이블 분리
ageless = features.copy()
ageless.remove('Age')
X2, y2 = data[ageless].values, data['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train2, X_test2, y_train2, y_test2, S_train2, S_test2 = train_test_split(X2, y2, S, test_size=0.20, random_state=0, stratify=y2)

# 분류 모델 학습
print("Training model...")
ageless_model = DecisionTreeClassifier().fit(X_train2, y_train2)
print("Model trained.")

# 이 모델을 Fairlearn의 공정성 대시보드에서 확인하고 나타내는 차이를 확인합니다.
FairlearnDashboard(sensitive_features=S_test2, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test2,
                   y_pred={"ageless_diabetes_model": ageless_model.predict(X_test2)})

대시보드에서 모델을 살펴봅니다.

*재현율*을 검토해 보면 부분 모집단 간의 모델 성능 차이가 감소했음을 확인할 수 있습니다. 하지만 이번에는 모델이 고령층 환자의 긍정 사례를 크게 '과소 예측'했기 때문에 전반적인 재현율 자체도 낮아졌습니다. 즉, **Age**를 학습 과정에서 기능으로 사용하지 않았지만 이번에도 환자의 연령에 따라 모델의 당뇨병 예측 성능에서 다소 차이가 나타났습니다.

이 시나리오에서는 **Age** 기능만 제거하면 *재현율*의 차이는 약간 감소하지만 *정밀도*와 *정확도*의 차이는 더 커집니다. 이처럼 기계 학습 모델에 공정성을 적용하기란 매우 까다롭습니다. 그러므로 특정 상황에서 *공정성*의 의미를 명확하게 정의하여 해당 정의에 맞게 모델을 최적화해야 합니다.

## 모델을 등록하고 작업 영역에 대시보드 데이터 업로드

이 Notebook의 앞부분에서는 모델을 학습시키고 로컬에서 대시보드를 검토했습니다. 이 모델을 Azure Machine Learning 작업 영역에 등록한 다음 실험을 만들어 대시보드 데이터를 기록하면 공정성 분석 과정을 추적하여 결과를 공유할 수 있으므로 유용할 수 있습니다.

먼저 **Age**가 기능으로 포함된 원래 모델을 등록합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
from azureml.core import Workspace, Experiment, Model
import joblib
import os

# 저장된 구성 파일에서 Azure ML 작업 영역 로드
ws = Workspace.from_config()
print('Ready to work with', ws.name)

# 학습된 모델 저장
model_file = 'diabetes_model.pkl'
joblib.dump(value=diabetes_model, filename=model_file)

# 모델 등록
print('Registering model...')
registered_model = Model.register(model_path=model_file,
                                  model_name='diabetes_classifier',
                                  workspace=ws)
model_id= registered_model.id


print('Model registered.', model_id)

이제 FairLearn 패키지를 사용하여 모델 하나 이상에 사용할 이진 분류 그룹 메트릭 집합을 만든 다음, Azure Machine Learning 실험을 사용하여 메트릭을 업로드할 수 있습니다.

> **참고**: 실험을 실행하려면 시간이 다소 걸릴 수 있습니다. 실험이 완료되면 대시보드 데이터가 다운로드되어 표시되므로 실험이 정상적으로 업로드되었음을 확인할 수 있습니다.

In [ ]:
from fairlearn.metrics._group_metric_set import _create_group_metric_set
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

#  공정성을 평가하려는 모델의 사전 만들기 
sf = { 'Age': S_test.Age}
ys_pred = { model_id:diabetes_model.predict(X_test) }
dash_dict = _create_group_metric_set(y_true=y_test,
                                    predictions=ys_pred,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')

exp = Experiment(ws, 'mslearn-diabetes-fairness')
print(exp)

run = exp.start_logging()

# Azure Machine Learning에 대시보드 업로드
try:
    dashboard_title = "Fairness insights of Diabetes Classifier"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
    print(downloaded_dict)
finally:
    run.complete()

위의 코드는 실험이 정상적으로 완료되었음을 확인하기 위해 실험에서 생성된 메트릭을 다운로드합니다. 실험에 메트릭을 업로드하면 제공되는 가장 큰 이점은 Azure Machine Learning Studio에서 FairLearn 대시보드를 확인할 수 있다는 것입니다.

아래 셀의 명령을 실행하여 실험 세부 정보를 확인합니다. 그런 다음 위젯에서 **실행 세부 정보 보기** 링크를 클릭하여 Azure Machine Learning Studio에서 실행 세부 정보를 확인합니다. 그런 다음 실험 실행의 **공정성** 탭에서 대시보드를 확인합니다. 이 탭에 표시되는 대시보드의 동작 방식도 이 Notebook의 앞부분에서 확인한 위젯과 동일합니다.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Azure Machine Learning Studio의 **모델** 페이지에서 모델을 선택하고 **공정성** 탭을 검토하는 방법으로 공정성 대시보드를 확인할 수도 있습니다. 그러면 조직에서는 학습시켜 등록하는 모델의 공정성 분석 로그를 유지 관리할 수 있습니다.

## 모델의 불공정성 완화

모델의 공정성을 분석한 후에는 FairLearn 패키지에서 지원하는 다양한 *완화* 기술 중 원하는 기술을 사용하여 예측 성능과 공정성의 균형이 적절하게 유지되는 모델을 찾을 수 있습니다.

이 연습에서는 **GridSearch** 기능을 사용합니다. 이 기능은 여러 모델을 학습시켜 데이터 세트 내의 중요 기능(여기서는 연령 그룹)에 대한 예측 성능 차이를 최소화합니다. 그리고 **EqualizedOdds** 패리티 제약 조건을 적용하여 모델을 최적화합니다. 이 제약 조건은 각 중요 기능 그룹에서 진양성 비율과 가양성 비율이 비슷한 모델 확인을 시도합니다. 

> *이 명령을 실행하려면 시간이 다소 걸릴 수 있습니다.*

In [ ]:
from fairlearn.reductions import GridSearch, EqualizedOdds
import joblib
import os

print('Finding mitigated models...')

# 여러 모델 학습시키기
sweep = GridSearch(DecisionTreeClassifier(),
                   constraints=EqualizedOdds(),
                   grid_size=20)

sweep.fit(X_train, y_train, sensitive_features=S_train.Age)
models = sweep.predictors_

# 모델을 저장하고 모델에서 예측 가져오기(비교를 위해 완화되지 않은 원래의 모델도 함께)
model_dir = 'mitigated_models'
os.makedirs(model_dir, exist_ok=True)
model_name = 'diabetes_unmitigated'
print(model_name)
joblib.dump(value=diabetes_model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
predictions = {model_name: diabetes_model.predict(X_test)}
i = 0
for model in models:
    i += 1
    model_name = 'diabetes_mitigated_{0}'.format(i)
    print(model_name)
    joblib.dump(value=model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
    predictions[model_name] = model.predict(X_test)


이제 FairLearn 대시보드를 사용하여 불공정성이 완화된 모델을 비교할 수 있습니다.

다음 셀의 명령을 실행한 후 마법사를 사용하여 **재현율**을 기준으로 **Age** 그룹을 시각화합니다.

In [ ]:
FairlearnDashboard(sensitive_features=S_test, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test,
                   y_pred=predictions)

모델은 산점도에 표시됩니다. 예측 결과(선택률)의 차이나 선택한 성능 메트릭(여기서는 *재현율*)의 차이를 측정하여 모델을 비교할 수 있습니다. 여기서는 선택률에서 차이가 나타납니다. 연령은 당뇨병에 실제로 영향을 *주는* 요인이며 환자의 연령층이 높을수록 긍정 사례도 많아지기 때문입니다. 모델의 예측 성능 차이를 확인해야 하므로 **재현율 차이** 측정 옵션을 선택합니다.

차트에 모델 클러스터가 표시됩니다. 전반적인 *재현율* 메트릭이 X축이고 재현율의 차이가 Y축입니다. 재현율이 높고 차이는 낮은 '이상적' 모델은 이 산점도의 오른쪽 아래 부분에 해당하는 모델이라 할 수 있습니다. 이처럼 특정 요구에 맞게 예측 성능과 공정성 간의 균형을 적절하게 선택한 다음 해당 모델을 선택하여 세부 정보를 확인할 수 있습니다.

여기서 고려해야 할 중요한 사항은 모델에 불공정성 완화 기능을 적용하는 경우 중요 기능 그룹에서 전반적인 예측 성능은 낮아지며 그룹 간에 예측 성능 차이도 나타납니다. 일반적으로는 모델이 모집단의 전체 세그먼트를 공정하게 예측하도록 하려는 경우 전반적인 예측 성능 저하는 다소 감수해야 합니다.

> **참고**: *정밀도* 메트릭에는 예측된 샘플이 없어서 정밀도가 0.0으로 설정된다는 경고가 표시될 수 있습니다. 이 경고는 무시해도 됩니다.

## Azure Machine Learning에 완화 대시보드 메트릭 업로드

앞에서와 마찬가지로 완화 실험을 추적하기 위해 메트릭을 업로드할 수 있습니다. 이렇게 하려는 경우 다음 단계를 수행하면 됩니다.

1. GridSearch 프로세스에서 확인된 모델을 등록합니다.
2. 모델의 성능 및 차이 메트릭을 계산합니다.
3. Azure Machine Learning 실험에서 메트릭을 업로드합니다.

In [ ]:
# 모델 등록
registered_model_predictions = dict()
for model_name, prediction_data in predictions.items():
    model_file = os.path.join(model_dir, model_name + ".pkl")
    registered_model = Model.register(model_path=model_file,
                                      model_name=model_name,
                                      workspace=ws)
    registered_model_predictions[registered_model.id] = prediction_data

#  모든 모델에 대한 연령 기능을 기반으로 이진 분류를 위해 설정된 그룹 메트릭 만들기
sf = { 'Age': S_test.Age}
dash_dict = _create_group_metric_set(y_true=y_test,
                                     predictions=registered_model_predictions,
                                     sensitive_features=sf,
                                     prediction_type='binary_classification')

exp = Experiment(ws, "mslearn-diabetes-fairness")
print(exp)

run = exp.start_logging()
RunDetails(run).show()

# Azure Machine Learning에 대시보드 업로드
try:
    dashboard_title = "Fairness Comparison of Diabetes Models"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))
finally:
    run.complete()

> **참고**: 예측된 샘플이 없어서 정밀도가 0.0으로 설정된다는 경고가 표시될 수 있습니다. 이 경고는 무시해도 됩니다.


실험 실행이 완료되면 위젯의 **실행 세부 정보 보기** 링크를 클릭하여 Azure Machine Learning Studio에서 실행을 확인하고 **공정성** 탭에서 FairLearn 대시보드를 확인합니다.